# Gene Analysis, heatmaps - gget cbio

In [1]:
# try:
#     import varseek as vk
# except ImportError:
#     print("varseek not found, installing...")
#     !pip install -U -q varseek
# try:
#     import RLSRWP_2025
# except ImportError:
#     print("RLSRWP_2025 not found, installing...")
#     !pip install -q git+https://github.com/pachterlab/RLSRWP_2025.git
# try:
#     import gget
# except ImportError:
#     print("gget not found, installing...")
#     !pip install -U -q gget

In [2]:
import os
import anndata as ad
import pandas as pd
import gget
import subprocess
import ast

from varseek.utils import convert_mutation_cds_locations_to_cdna, get_ensembl_gene_id_bulk

RLSRWP_2025_dir = os.path.dirname(os.path.abspath(""))  # if this notebook resides in RLSRWP_2025/notebooks/0_data_download.ipynb, then this retrieves RLSRWP_2025

In [3]:
n = 3
sorting_column = "vcrs_count"  # "vcrs_count" (sort by variant), "gene_count" (sort by gene), etc

vk_count_out_dir = os.path.join(RLSRWP_2025_dir, "data", "vk_count_out")
adata_path = os.path.join(vk_count_out_dir, "adata_cleaned.h5ad")

out_dir = os.path.join(vk_count_out_dir, "analysis", "gget_cbio")

reference_dir = os.path.join(RLSRWP_2025_dir, "data", "reference")
cbio_data_dir = os.path.join(reference_dir, "cbio_data")
cosmic_csv = os.path.join(reference_dir, "cosmic", "CancerMutationCensus_AllData_Tsv_v101_GRCh37", "CancerMutationCensus_AllData_v101_GRCh37_mutation_workflow.csv")
sequences_cds = os.path.join(reference_dir, "ensembl_grch37_release93", "Homo_sapiens.GRCh37.cds.all.fa")
sequences_cdna = os.path.join(reference_dir, "ensembl_grch37_release93", "Homo_sapiens.GRCh37.cdna.all.fa")

In [ ]:
#!!! DOWNLOAD

In [4]:
os.makedirs(out_dir, exist_ok=True)
adata = ad.read_h5ad(adata_path)

# For simplicity, filter out all mutants with ambiguous identity
adata = adata[:, adata.var[~adata.var['vcrs_header'].str.contains(';', na=False)].index].copy()

adata.var.head()

,vcrs_header,vcrs_count,gene_name
0,ENST00000391429:c.194C>T,0.0,BHLHA9
1,ENST00000391429:c.128C>A,0.0,BHLHA9
2,ENST00000391429:c.257C>A,0.0,BHLHA9
3,ENST00000391429:c.100G>A,0.0,BHLHA9
4,ENST00000391429:c.99G>A,0.0,BHLHA9


In [5]:
adata_var_columns = adata.var.columns

# download cosmic and cdna
if not os.path.exists(sequences_cdna):
    print("Downloading cDNA")
    reference_cdna_dir = os.path.dirname(sequences_cdna) if os.path.dirname(sequences_cdna) else "."
    gget_ref_command = ["gget", "ref", "-w", "cdna", "-r", "93", "--out_dir", reference_cdna_dir, "-d", "human_grch37"]
    subprocess.run(gget_ref_command, check=True)
    subprocess.run(["gunzip", f"{sequences_cdna}.gz"], check=True)
if not os.path.exists(sequences_cds):
    print("Downloading CDS")
    reference_cds_dir = os.path.dirname(sequences_cds) if os.path.dirname(sequences_cds) else "."
    gget_ref_command = ["gget", "ref", "-w", "cds", "-r", "93", "--out_dir", reference_cds_dir, "-d", "human_grch37"]
    subprocess.run(gget_ref_command, check=True)
    subprocess.run(["gunzip", f"{sequences_cds}.gz"], check=True)

if not os.path.exists(cosmic_csv):
    print("Downloading COSMIC")
    reference_out_dir_cosmic = os.path.dirname(os.path.dirname(cosmic_csv))
    gget.cosmic(
        None,
        grch_version=37,
        cosmic_version=101,
        out=reference_out_dir_cosmic,
        mutation_class="cancer",
        download_cosmic=True,
    )

cosmic_df = pd.read_csv(cosmic_csv, nrows=2)

if "mutation_cdna" not in cosmic_df.columns:
    print("Converting CDS to cDNA in COSMIC")
    _, _ = convert_mutation_cds_locations_to_cdna(input_csv_path=cosmic_csv, output_csv_path=cosmic_csv, cds_fasta_path=sequences_cds, cdna_fasta_path=sequences_cdna, verbose=True)

cosmic_df = pd.read_csv(cosmic_csv, usecols=["seq_ID", "mutation_cdna", "gene_name"])
cosmic_df["vcrs_header"] = cosmic_df["seq_ID"] + ":" + cosmic_df["mutation_cdna"]

if "gene_name" not in adata.var.columns:
    adata.var = adata.var.merge(cosmic_df[["vcrs_header", "gene_name"]], on="vcrs_header", how="left")

if sorting_column == "gene_count" and "gene_count" not in adata.var.columns:
    filtered_var = adata.var[adata.var["vcrs_count"] > 0]
    gene_counts = filtered_var["gene_name"].value_counts()
    adata.var["gene_count"] = adata.var["gene_name"].map(gene_counts).fillna(0).astype(int)
    del filtered_var

if set(adata_var_columns) != set(adata.var.columns):
    adata.write_h5ad(adata_path)

In [6]:
adata.var = adata.var.sort_values(by=sorting_column, ascending=False)
adata.var.head()

,vcrs_header,vcrs_count,gene_name
691684,ENST00000362079:c.760G>A,10166.0,MT-CO3
2675243,ENST00000393099:c.583G>A,7259.0,RPL13
2021384,ENST00000251453:c.455C>T,5370.0,RPS16
4543187,ENST00000423316:c.2076C>T,4317.0,EEF1D
2685771,ENST00000593646:c.53G>A,2736.0,AC040977.1


In [7]:
top_variants = adata.var["vcrs_header"][:n].tolist()
top_transcripts = [header.split(":")[0] for header in top_variants]
top_transcripts = list(dict.fromkeys(top_transcripts))  # remove duplicates while preserving order
print(top_transcripts)

['ENST00000362079', 'ENST00000393099', 'ENST00000251453']


### While I could simply get the gene names from adata.var, it is more reliable to get them from Ensembl transcript IDs

In [8]:
transcript_to_gene_mapping = get_ensembl_gene_id_bulk(top_transcripts)
top_genes = [transcript_to_gene_mapping[transcript] for transcript in top_transcripts]
print(top_genes)

['ENSG00000198938', 'ENSG00000167526', 'ENSG00000105193']


### Find studies in the cBioPortal database that focus on the organ of interest - in this case, the skin

In [9]:
skin_studies = !gget cbio search skin
skin_studies = ast.literal_eval("".join(skin_studies))  # list-like string --> list
skin_studies

['mel_mskimpact_2020',
 'normal_skin_fibroblast_2024',
 'normal_skin_keratinocytes_2024',
 'normal_skin_melanocytes_2024',
 'skcm_broad',
 'skcm_broad_brafresist_2012',
 'skcm_broad_dfarber',
 'skcm_dfci_2015',
 'skcm_mskcc_2014',
 'skcm_tcga',
 'skcm_tcga_gdc',
 'skcm_tcga_pan_can_atlas_2018',
 'skcm_tcga_pub_2015',
 'skcm_vanderbilt_mskcc_2015',
 'skcm_yale']

In [10]:
# list --> space-separated string
top_genes = " ".join(top_genes)
skin_studies = " ".join(skin_studies)

In [ ]:
stratification = "cancer_type_detailed"
counts = "mutation_occurrences"
file_name = f"top_{n}_{sorting_column}_{stratification}_{counts}.png"
!gget cbio plot -s {skin_studies} -g {top_genes} -st {stratification} -vt {counts} -nc -dd {cbio_data_dir} -fd {out_dir} -fn {file_name}

01:09:51 - INFO - Downloading data
01:09:51 - INFO - File /Users/joeyrich/Desktop/local/RLSRWP_2025/data/reference/cbio_data/mel_mskimpact_2020/mutations.txt already exists, skipping.
01:09:51 - INFO - File /Users/joeyrich/Desktop/local/RLSRWP_2025/data/reference/cbio_data/mel_mskimpact_2020/cna.txt already exists, skipping.
01:09:51 - INFO - File /Users/joeyrich/Desktop/local/RLSRWP_2025/data/reference/cbio_data/mel_mskimpact_2020/sv.txt already exists, skipping.
01:09:51 - INFO - File /Users/joeyrich/Desktop/local/RLSRWP_2025/data/reference/cbio_data/mel_mskimpact_2020/clinical_patient.txt already exists, skipping.
01:09:51 - INFO - File /Users/joeyrich/Desktop/local/RLSRWP_2025/data/reference/cbio_data/mel_mskimpact_2020/clinical_sample.txt already exists, skipping.
01:09:51 - INFO - Downloaded all data for study mel_mskimpact_2020
01:09:51 - ERROR - Failed to download mutations data for study normal_skin_fibroblast_2024
01:09:51 - ERROR - Failed to download cna data for study norma

In [12]:
stratification = "study_id"
counts = "mutation_occurrences"
file_name = f"top_{n}_{sorting_column}_{stratification}_{counts}.png"
!gget cbio plot -s {skin_studies} -g {top_genes} -st {stratification} -vt {counts} -sh -nc -dd {cbio_data_dir} -fd {out_dir} -fn {file_name}

01:15:42 - INFO - Downloading data
01:15:42 - INFO - File /Users/joeyrich/Desktop/local/RLSRWP_2025/data/reference/cbio_data/mel_mskimpact_2020/mutations.txt already exists, skipping.
01:15:42 - INFO - File /Users/joeyrich/Desktop/local/RLSRWP_2025/data/reference/cbio_data/mel_mskimpact_2020/cna.txt already exists, skipping.
01:15:42 - INFO - File /Users/joeyrich/Desktop/local/RLSRWP_2025/data/reference/cbio_data/mel_mskimpact_2020/sv.txt already exists, skipping.
01:15:42 - INFO - File /Users/joeyrich/Desktop/local/RLSRWP_2025/data/reference/cbio_data/mel_mskimpact_2020/clinical_patient.txt already exists, skipping.
01:15:42 - INFO - File /Users/joeyrich/Desktop/local/RLSRWP_2025/data/reference/cbio_data/mel_mskimpact_2020/clinical_sample.txt already exists, skipping.
01:15:42 - INFO - Downloaded all data for study mel_mskimpact_2020
01:15:42 - ERROR - Failed to download mutations data for study normal_skin_fibroblast_2024
01:15:43 - ERROR - Failed to download cna data for study norma

In [13]:
stratification = "sample"
counts = "mutation_occurrences"
file_name = f"top_{n}_{sorting_column}_{stratification}_{counts}.png"
!gget cbio plot -s {skin_studies} -g {top_genes} -st {stratification} -vt {counts} -sh -nc -dd {cbio_data_dir} -fd {out_dir} -fn {file_name}

01:18:16 - INFO - Downloading data
01:18:16 - INFO - File /Users/joeyrich/Desktop/local/RLSRWP_2025/data/reference/cbio_data/mel_mskimpact_2020/mutations.txt already exists, skipping.
01:18:16 - INFO - File /Users/joeyrich/Desktop/local/RLSRWP_2025/data/reference/cbio_data/mel_mskimpact_2020/cna.txt already exists, skipping.
01:18:16 - INFO - File /Users/joeyrich/Desktop/local/RLSRWP_2025/data/reference/cbio_data/mel_mskimpact_2020/sv.txt already exists, skipping.
01:18:16 - INFO - File /Users/joeyrich/Desktop/local/RLSRWP_2025/data/reference/cbio_data/mel_mskimpact_2020/clinical_patient.txt already exists, skipping.
01:18:16 - INFO - File /Users/joeyrich/Desktop/local/RLSRWP_2025/data/reference/cbio_data/mel_mskimpact_2020/clinical_sample.txt already exists, skipping.
01:18:16 - INFO - Downloaded all data for study mel_mskimpact_2020
01:18:16 - ERROR - Failed to download mutations data for study normal_skin_fibroblast_2024
01:18:17 - ERROR - Failed to download cna data for study norma